In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Combining

In [2]:
ptdb_normal=pd.read_csv("heartbeat/ptbdb_normal.csv",header=None)
ptdb_abnormal=pd.read_csv("heartbeat/ptbdb_abnormal.csv",header=None)
ptdb_normal.iloc[:,187]=0
ptdb_abnormal.iloc[:,187]=1

mitbih_train=pd.read_csv("heartbeat/mitbih_train.csv",header=None)
mitbih_test=pd.read_csv("heartbeat/mitbih_test.csv",header=None)
mitbih_train.replace([1, 2, 3,4], 1,inplace=True)
mitbih_test.replace([1, 2, 3,4], 1,inplace=True)

ptdb_normal.shape,ptdb_abnormal.shape,mitbih_train.shape,mitbih_test.shape

((4046, 188), (10506, 188), (87554, 188), (21892, 188))

## for anomaly

In [3]:
def save_csv(normal_train,normal_valid,patient_valid,path):
    
    df_normal_train=pd.DataFrame(normal_train)
    df_normal_valid=pd.DataFrame(normal_valid)
    df_patient_valid=pd.DataFrame(patient_valid)


    df_normal_train.to_csv(path+"/normal_train.csv",index=False)
    df_normal_valid.to_csv(path+"/normal_valid.csv",index=False)
    df_patient_valid.to_csv(path+"/patient_valid.csv",index=False)

In [4]:
normal=ptdb_normal.values
normal_train=normal[120:]
normal_valid=normal[:120]
patient_valid=ptdb_abnormal.values[:120]
print(normal_train.shape,normal_valid.shape,patient_valid.shape)

save_csv(normal_train,normal_valid,patient_valid,"./input/ecgdatasets/anomaly/ptdb")

(3926, 188) (120, 188) (120, 188)


In [37]:
normal_train=mitbih_train[mitbih_train[187]==0][:4000]
normal_valid=mitbih_test[mitbih_test[187]==0][:120]
patient_valid=mitbih_test.values[mitbih_test[187]==1][:120]

print(normal_train.shape,normal_valid.shape,patient_valid.shape)

save_csv(normal_train,normal_valid,patient_valid,"./input/ecgdatasets/anomaly/mit")

(4000, 188) (120, 188) (120, 188)


In [38]:
#amplyfying signals for mit dataset
import random

for i in range(len(normal_valid)):
    d=normal_valid.iloc[i,:187]
    rand=random.random()
    if(rand>0.5):
        #increased heart rate
        c=random.random()
        for j in range(187):
            if(int(j/c)>=187):
                for k in range(j,187):
                    d[k]=0
                break
            d[j]=d[int(j/c)]
    else:
        #amplify signal
        c=random.randint(1,10)
        for j in range(187):
            d[j]=d[j]*2
            
    normal_valid.iloc[i,:187]=d
    normal_valid.iloc[:,-1]=1

In [39]:
new_normal_valid=mitbih_test[mitbih_test[187]==0][:120]
save_csv(normal_train,new_normal_valid,normal_valid,"./input/ecgdatasets/anomaly/mit_amplified")

## for classification

In [41]:
df_train1=mitbih_train[mitbih_train.iloc[:,-1]==1].iloc[:3000,:]
df_train2=mitbih_train[mitbih_train.iloc[:,-1]==0].iloc[:3000,:]
df_train=pd.DataFrame(np.vstack((df_train1.values,df_train2.values)))
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test1=mitbih_test[mitbih_test.iloc[:,-1]==1].iloc[:100,:]
df_test2=mitbih_test[mitbih_test.iloc[:,-1]==0].iloc[:100,:]
df_test=pd.DataFrame(np.vstack((df_test1.values,df_test2.values)))
df_test = df_test.sample(frac=1).reset_index(drop=True)


print(df_train.shape,df_test.shape)

(6000, 188) (200, 188)


In [42]:
df_train.iloc[:,-1].value_counts()

0.0    3000
1.0    3000
Name: 187, dtype: int64

In [43]:
df_train.to_csv("./input/ecgdatasets/classification/mit/train.csv",index=False)
df_test.to_csv("./input/ecgdatasets/classification/mit/test.csv",index=False)

In [44]:
#amplyfying signals for mit dataset
import random

for i in range(len(df_test2)):
    d=df_test2.iloc[i,:187]
    rand=random.random()
    if(rand>0.5):
        #increased heart rate
        c=random.random()
        for j in range(187):
            if(int(j/c)>=187):
                for k in range(j,187):
                    d[k]=0
                break
            d[j]=d[int(j/c)]
    else:
        #amplify signal
        c=random.randint(1,10)
        for j in range(187):
            d[j]=d[j]*2
            
    df_test2.iloc[i,:187]=d

df_test2.iloc[:,-1]=1
df_test3=mitbih_test[mitbih_test.iloc[:,-1]==0].iloc[:100,:]
df_test=pd.DataFrame(np.vstack((df_test3.values,df_test2.values)))
df_test = df_test.sample(frac=1).reset_index(drop=True)


print(df_test.shape)

(200, 188)


In [45]:
df_train.to_csv("./input/ecgdatasets/classification/mit_amplified/train.csv",index=False)
df_test.to_csv("./input/ecgdatasets/classification/mit_amplified/test.csv",index=False)

In [27]:
p1=ptdb_normal.iloc[:3000,:]
p2=ptdb_abnormal.iloc[:3000,:]

df_train = pd.DataFrame(np.vstack((p1.values,p2.values)))
df_train = df_train.sample(frac=1).reset_index(drop=True)

p1=ptdb_normal.iloc[3000:3100,:]
p2=ptdb_abnormal.iloc[3000:3100,:]

df_test = pd.DataFrame(np.vstack((p1.values,p2.values)))
df_test = df_test.sample(frac=1).reset_index(drop=True)

print(df_train.shape,df_test.shape)

(6000, 188) (200, 188)


In [28]:
df_train.iloc[:,-1].value_counts()

0.0    3000
1.0    3000
Name: 187, dtype: int64

In [29]:
df_train.to_csv("./input/ecgdatasets/classification/ptdb/train.csv",index=False)
df_test.to_csv("./input/ecgdatasets/classification/ptdb/test.csv",index=False)